In [1]:
import pandas as pd
import numpy as np


### Data Cleaning , Manipulating & Transforming

In [240]:
data=pd.read_csv(path)
data.drop('Unnamed: 0',axis=1,inplace=True)

bowled=[]
for x in range(len(data)):
    if (data['caught'].fillna('None')[x]=='None') & (data['bowled_by'][x]!='None'):
        bowled.append(1)
    else:
        bowled.append(0)
        
data['is_bowled']=bowled

bowled_count=data.groupby(['bowled_by','match_id'])['is_bowled'].sum().reset_index()
bowled_count=bowled_count[bowled_count['is_bowled']!=0].reset_index().drop('index',axis=1)
bowled_count.columns=['bowler_name','match_id','bowled_count']

data=pd.merge(data,bowled_count,how='left',left_on=['match_id','popular_name'],right_on=['match_id','bowler_name'])
data=data.drop('bowler_name',axis=1)
data['bowled_count']=data['bowled_count'].fillna(0)

lbw=data.groupby(['match_id','lbw_by']).agg(lbw_count=('lbw_by', 'count')).reset_index()
lbw=lbw[lbw["lbw_by"].str.contains("None") == False].reset_index().drop('index',axis=1)
lbw.columns=['match_id','lbw','lbw_count']

data=pd.merge(data,lbw,how='left',left_on=['match_id','popular_name'],right_on=['match_id','lbw'])
data=data.drop('lbw',axis=1)
data['lbw_count']=data['lbw_count'].fillna(0)


df_caught=data.groupby(['match_id','caught']).agg(catches=('caught','count')).reset_index()
df_caught=df_caught[(df_caught["caught"].str.contains("None") | df_caught["caught"].str.contains("dnb")) | df_caught["caught"].str.contains("not out") == False].reset_index().drop('index',axis=1)
df_caught.columns=['match_id','caught_by','catches']

data=pd.merge(data,df_caught,how='left',left_on=['match_id','popular_name'],right_on=['match_id','caught_by'])
data=data.drop('caught_by',axis=1)

df_dh=data.groupby(['match_id','run_out_dh']).agg(direct_hit=('run_out_dh','count')).reset_index()

df_dh=df_dh[df_dh['run_out_dh'].str.contains('None')| df_dh['run_out_dh'].str.contains('sub ')==False].reset_index().drop('index',axis=1)
df_dh.columns=['match_id','run_out_by','direct_hit']


df_as1=data.groupby(['match_id','run_out_1']).agg(ro_assist=('run_out_1','count')).reset_index()
df_as1=df_as1[df_as1['run_out_1'].str.contains('None')| df_as1['run_out_1'].str.contains('sub ')==False].reset_index().drop('index',axis=1)
df_as1.columns=['match_id','run_out_by','ro_assist_1']

df_as2=data.groupby(['match_id','run_out_2']).agg(ro_assist=('run_out_2','count')).reset_index()
df_as2=df_as2[df_as2['run_out_2'].str.contains('None')| df_as2['run_out_2'].str.contains('sub ')==False].reset_index().drop('index',axis=1)
df_as2.columns=['match_id','run_out_by','ro_assist_2']



df_ro=pd.concat([df_dh,df_as1,df_as2],axis=0).fillna(0).reset_index().drop('index',axis=1)

data=pd.merge(data,df_ro,how='left',left_on=['match_id','popular_name'],right_on=['match_id','run_out_by'])
data=data.drop('run_out_by',axis=1)

catch_3=data.groupby(['match_id','caught'])['catches'].sum().reset_index()
catch_3=catch_3[catch_3['caught'].str.contains('dnb')|catch_3['caught'].str.contains('None')|catch_3['caught'].str.contains('sub ')| catch_3["caught"].str.contains("not out")==False]
catch_3=catch_3[catch_3['catches']>=3].reset_index().drop('index',axis=1)
catch_3.columns=['match_id','caught_by','3_catches']


data=pd.merge(data,catch_3,how='left',left_on=['match_id','popular_name'],right_on=['match_id','caught_by'])
data=data.drop('caught_by',axis=1)

stump=data.groupby(['match_id','stumped_out_by']).agg(stumped=('stumped_out_by','count')).reset_index()
stump[stump['stumped_out_by'].str.contains('None')==False].reset_index().drop('index',axis=1)
stump.columns=['match_id','so_by','stump_outs']


data=pd.merge(data,stump,how='left',left_on=['match_id','popular_name'],right_on=['match_id','so_by'])
data=data.drop('so_by',axis=1)


data['SR']=(data['SR'].replace(regex=True,to_replace=r'[^\d+\.\d+]',value=0.0)).astype('float')
data

### Dream_11_points

In [272]:

dream11_points=pd.DataFrame()

dream11_points['player']=data['known_as']
dream11_points['match_id']=data['match_id']

dream11_points['In_Announced']=4

dream11_points['R_points']=data['R']*1
dream11_points['4s_points']=data['4s']*1
dream11_points['6s_points']=data['6s']*2
dream11_points['30_Run_points']=data['R'].apply(lambda x: 4 if (x>=30) & (x<50) else 0)
dream11_points['50_Run_points']=data['R'].apply(lambda x: 8 if (x>=50) & (x<100) else 0)
dream11_points['100_Run_points']=data['R'].apply(lambda x: 16 if x>=100 else 0)
dream11_points['SR_points']=data['SR'].apply(lambda x: 6 if x>170 else 4 if (x>150)&(x<=170) else 2 if (x>=130)&(x<=150) else -2 if (x>=60)&(x<=70) else -4 if (x>=50)&(x<60) else -6)

duck_points=[]
econ_point=[]
for x in range(len(data)):
    if (data['bowler'][x] !=1) & (data['R'][x] ==0) & (data['CB'][x]!='dnb'):
        duck_points.append(-2)
        dream11_points['SR_points'][x]=0
        
    elif (data['bowler'][x] ==1) & (data['R'][x]==0)  & (data['CB'][x]!='dnb'):
        duck_points.append(0)
        dream11_points['SR_points'][x]=0
        
    elif (data['B'][x]==0) & (data['CB'][x]!='dnb'):
        duck_points.append(0)
        dream11_points['SR_points'][x]=0
        
    elif (data['CB'][x]=='dnb'):
        duck_points.append(0)
        dream11_points['SR_points'][x]=0

    else:
        duck_points.append(0)
    
    if data['Overs'][x]>=2:
        econ_point.append(6 if data['ECON'][x]<5 else 4 if (data['ECON'][x]>=5) & (data['ECON'][x]<6) \
                          else 2 if (data['ECON'][x]>=6) & (data['ECON'][x]<=7) \
                          else -2 if (data['ECON'][x]>=10) & (data['ECON'][x]<=11) \
                         else -4 if (data['ECON'][x]>11) & (data['ECON'][x]<=12)\
                        else -6 if data['ECON'][x]>12 else 0
                         )
    else:
        econ_point.append(0)
    
        
dream11_points['duck_points']=duck_points

#bowling points

dream11_points['eco_points']=econ_point
dream11_points['Wicket_points']=data['Wickets'].apply(lambda x: x*25).fillna(0)
dream11_points['bowled']=data['bowled_count']*8
dream11_points['3wk_points']=data['Wickets'].apply(lambda x: 4 if x==3 else 0)
dream11_points['4wk_points']=data['Wickets'].apply(lambda x: 8 if x==4 else 0)
dream11_points['5wk_points']=data['Wickets'].apply(lambda x: 16 if x>=5 else 0)
dream11_points['Maiden_points']=data['Maidens']*12

#fielding points
dream11_points['catch_points']=data['catches']*8
dream11_points['3catch_points']=np.where(data['3_catches']>0,4,0)
dream11_points['stumping']=data['stump_outs']*12
dream11_points['ro_dh_points']=data['direct_hit']*12
dream11_points['ro_assist_1']=data['ro_assist_1']*6
dream11_points['ro_assist_2']=data['ro_assist_2']*6

dream11_points['total_points']=dream11_points.iloc[:,2:].sum(axis=1)
dream11_points.fillna(0,inplace=True)

dream11_points.sort_values(by=['match_id','total_points'],ascending=False)

dream11_points


<ipython-input-272-9e5fbeab69e9>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dream11_points['SR_points'][x]=0
<ipython-input-272-9e5fbeab69e9>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dream11_points['SR_points'][x]=0
<ipython-input-272-9e5fbeab69e9>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dream11_points['SR_points'][x]=0


,player,match_id,In_Announced,R_points,4s_points,6s_points,30_Run_points,50_Run_points,100_Run_points,SR_points,...,4wk_points,5wk_points,Maiden_points,catch_points,3catch_points,stumping,ro_dh_points,ro_assist_1,ro_assist_2,total_points
0,Rohit Sharma,1327508,4,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,2.0
1,Rishabh Pant,1327508,4,27,3,4,0,0,0,6,...,0,0,0.0,0.0,0,0.0,0.0,0.0,6.0,50.0
2,Shreyas Iyer,1327508,4,1,0,0,0,0,0,-6,...,0,0,0.0,0.0,0,0.0,0.0,6.0,0.0,5.0
3,Dinesh Karthik,1327508,4,46,4,8,4,0,0,6,...,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,72.0
4,Suryakumar Yadav,1327508,4,8,0,2,0,0,0,2,...,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,Sam Curran,1243388,4,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,4.0
1030,Jofra Archer,1243388,4,0,0,0,0,0,0,0,...,0,0,12.0,0.0,0,0.0,0.0,0.0,0.0,107.0
1031,Chris Jordan,1243388,4,0,0,0,0,0,0,0,...,0,0,0.0,16.0,0,0.0,0.0,0.0,0.0,47.0
1032,Adil Rashid,1243388,4,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,35.0


## Best 11 for each match


In [273]:
best_11=dream11_points.set_index('player').groupby('match_id')['total_points'].nlargest(11).reset_index()
best_11[best_11['match_id']==1327508]


,match_id,player,total_points
506,1327508,Rilee Rossouw,149.0
507,1327508,Quinton de Kock,106.0
508,1327508,Dwaine Pretorius,83.0
509,1327508,Deepak Chahar,74.0
510,1327508,Dinesh Karthik,72.0
511,1327508,Keshav Maharaj,62.0
512,1327508,David Miller,59.0
513,1327508,Wayne Parnell,52.0
514,1327508,Rishabh Pant,50.0
515,1327508,Lungi Ngidi,48.0
